In [1]:
import os

import datasets
import torch
from transformers import pipeline

In [2]:
df = datasets.load_dataset('kreimben/leetcode_with_youtube_captions')['train']
df = df.to_pandas()
df.sample(3)

,cc_content,id,thumbnail,title_x,question_content,java,c++,python,javascript,title_y,tag,level,success_rate,total_submission,total_accepted,question_likes,question_dislikes,question_hints,similar_question_ids
15967,alright guys welcome back to study plan alrig...,303,https://i.ytimg.com/vi/Ah1M05f0zug/sddefault.jpg,Range Sum Query - Immutable,"Given an integer array `nums`, handle multiple...",\n ```java\nclass NumArray {\n int[] pSu...,\n ```cpp\n#include <vector>\n\nclass NumAr...,\n ```python\nclass NumArray:\n\n def __...,\n ```javascript\nclass NumArray {\n con...,Range Sum Query - Immutable,"Array,Design,Prefix Sum",Easy,55.2,628000.0,346813.0,1928.0,1641.0,None,"304,307,325"
9093,okay let's talk about path sum too so okay le...,113,https://i.ytimg.com/vi/yKuc4CM1iQo/sddefault.jpg,Path Sum II,Given the `root` of a binary tree and an integ...,\n ```java\nimport java.util.List;\nimport ...,\n ```cpp\nstruct TreeNode {\n int val;\...,\n ```python\nclass TreeNode:\n def __in...,\n ```javascript\nclass TreeNode {\n con...,Path Sum II,"Backtracking,Tree,Depth-First Search,Binary Tree",Medium,53.9,1052748.0,567301.0,4603.0,109.0,None,"112,257,437,666,2217"
10434,hey everyone welcome back to another hey ever...,1816,https://i.ytimg.com/vi_webp/jLqIDIOXGgI/maxres...,Truncate Sentence,A **sentence** is a list of words that are sep...,\n ```java\nclass Solution {\n public St...,\n ```cpp\n#include <string>\n#include <sst...,"\n ```python\ndef truncateSentence(s: str, ...",\n ```javascript\nfunction truncateSentence...,Lowest Common Ancestor of a Binary Tree IV,"Tree,Depth-First Search,Binary Tree",Medium,79.4,25459.0,20221.0,283.0,9.0,"Starting from the root, traverse the left and ...","235,236,1218,1780,1790,1816"


In [3]:
# example = df[df['id'] == 1]['cc_content']
# example.sample(3)

In [4]:
# N = len(example)
# idx = random.randint(0, N)
# 
# example_yt_cc = example.values[idx]
# example_yt_cc

In [5]:
# Load the HF pipeline using Gemma 2B from Kaggle
pipe = pipeline(
    "text-generation",
    model='bert-large-uncased',
    model_kwargs={"torch_dtype": torch.float16},
    device='cuda',
    max_new_tokens=512,
    token=os.getenv('HF_API')
)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [6]:
prompt = """
You are a teacher who teaches coding tests to users.
You should explain the algorithm for the coding test problem to the user in detail.
separate the steps but keep it detailed.
let's think step by step.
"""

In [17]:
def summarize_example(text):
    global prompt
    max_input_length = 200  # Reducing input length for stricter truncation
    max_output_length = 512

    chunks = [text[i:i + max_input_length] for i in range(0, len(text), max_input_length)]

    summaries = []
    for chunk in chunks:
        input_text = f"{prompt}\n\n{chunk}"
        inputs = pipe.tokenizer(input_text, truncation=True, max_length=max_input_length, return_tensors="pt")

        # Ensure compatibility between output length and input constraints
        max_new_tokens = min(max_output_length, max_input_length - len(inputs['input_ids'][0]))  

        outputs = pipe(
            input_text,
            do_sample=True,
            temperature=0.1,
            top_k=5,
            top_p=0.3,
            max_new_tokens=max_new_tokens,
            pad_token_id=pipe.tokenizer.eos_token_id,
        )

        summary = outputs[0]["generated_text"]
        summaries.append(summary)

    return " ".join(summaries)


In [ ]:
df['summary'] = df['cc_content'].apply(summarize_example)

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\aksid\AppData\Local\Programs\Python\Python310\lib\logging\__init__.py", line 1100, in emit
    msg = self.format(record)
  File "C:\Users\aksid\AppData\Local\Programs\Python\Python310\lib\logging\__init__.py", line 943, in format
    return fmt.format(record)
  File "C:\Users\aksid\AppData\Local\Programs\Python\Python310\lib\logging\__init__.py", line 678, in format
    record.message = record.getMessage()
  File "C:\Users\aksid\AppData\Local\Programs\Python\Python310\lib\logging\__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\Users\aksid\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\aksid\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\User